# System Model V3 - Debt Market Model

The purpose of this notebook is to simulate a single run of the full CDP and APT system model, using a stochastic Ethereum price and liquidity demand process as a driver.


For this simulation run, we will be asking the following question: does market price decline without controller?

In order to test this, we override the default paramers seen in [init.py](models/system_model_v3/model/params/init.py) with the following and perform a parameter sweep:

```
params_update = {
    'controller_enabled': [False],
    'liquidity_demand_shock': [False],
    'rescale_target_price': [True, False], 
    'arbitrageur_considers_liquidation_ratio': [True,False],
}
```


# Setup and Dependencies

In [1]:
# Set project root folder, to enable importing project files from subdirectories
from pathlib import Path
import os

path = Path().resolve()
root_path = str(path).split('notebooks')[0]
os.chdir(root_path)

# Force reload of project modules, sometimes necessary for Jupyter kernel
%load_ext autoreload
%autoreload 2

# Display cadCAD version for easy debugging
%pip show cadCAD

Name: cadCAD
Version: 0.4.23
Summary: cadCAD: a differential games based simulation software package for research, validation, and         Computer Aided Design of economic systems
Home-page: https://github.com/cadCAD-org/cadCAD
Author: Joshua E. Jodesty
Author-email: joshua@block.science
License: LICENSE.txt
Location: /home/aclarkdata/repos/reflexer/venv/lib/python3.8/site-packages
Requires: fn, pandas, pathos, funcy
Required-by: cadCAD-machine-search
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Import all shared dependencies and setup
from shared import *
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# Environmental Processes

In [ ]:
# Import the ETH price stochastic dataset
from models.system_model_v3.model.params.init import eth_price_df

In [ ]:
eth_price_df

In [ ]:
time = []
for i in range(0,len(eth_price_df)):
    time.append(i)

In [ ]:
# Plot the ETH price for the first MC run dataset
sns.lineplot(data=eth_price_df, x=time, y="0",label='Generated Eth price')
plt.title('Generated Eth price')
plt.legend(loc="upper left")
plt.show()

# Model Configuration

In [ ]:
from models.system_model_v3.model.state_variables.init import state_variables

state_variables.update({})

In [ ]:
from models.system_model_v3.model.params.init import params


# Update the default parameter values
# params_update = {
#     'controller_enabled': [True],
#     'kp': [2.000000e-07], # proportional term for the stability controller: units 1/USD
#     'ki': [-5.000000e-09], # integral term for the stability controller: units 1/(USD*seconds)
# }
params_update = {
    'controller_enabled': [False],
    'liquidity_demand_shock': [False],
    'rescale_target_price': [True,False],
    'arbitrageur_considers_liquidation_ratio': [True,False]
}

params.update(params_update)


# Simulation Execution

In [ ]:
# Set the number of simulation timesteps, with a maximum of `len(eth_price_df) - 1`
SIMULATION_TIMESTEPS = len(eth_price_df) - 1
SIMULATION_TIMESTEPS

In [ ]:
# Create a wrapper for the model simulation, and update the existing parameters and initial state
system_simulation = ConfigWrapper(system_model_v3, T=range(SIMULATION_TIMESTEPS), M=params, initial_state=state_variables)

In [ ]:
del configs[:] # Clear any prior configs
(simulation_result, _tensor_field, _sessions) = run(system_simulation, drop_midsteps=False) # Run the simulation

# Simulation Analysis

In [ ]:
# Add new columns to dataframe
simulation_result = simulation_result.assign(eth_collateral_value = simulation_result.eth_collateral * simulation_result.eth_price)
simulation_result['collateralization_ratio'] = (simulation_result.eth_collateral * simulation_result.eth_price) / (simulation_result.principal_debt * simulation_result.target_price)

# Update dataframe display settings
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)

simulation_result

## Save simulation

In [ ]:
# Save the simulation result to a pickle file, for backup - this will be overwritten on the next simulation
#simulation_result.to_pickle(f'./exports/system_model_v3/results.pickle')
#simulation_result.to_pickle('notebooks/system_model_v3/results.pickle')

'''
1. Query: does market price decline without controller?
 ---
   - Run simulation with:
     - ETH price shock OFF
     - Liquidity shocks OFF
     - Controller OFF
     - Arbitrageur considers liquidation ratio:
       - ON
       - OFF
     - Target price rescaled:
       - ON
       - OFF
'''
simulation_result.to_pickle('notebooks/system_model_v3/query_1_results.pickle')

# Simulation Analysis

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# Load the simulation result from a pickle file, specifying past results when necessary
#simulation_result = pd.read_pickle('notebooks/analysis/saved_results/v3_results.pickle')

# Drop the simulation midsteps - the substeps that aren't used for generating plots
df = drop_dataframe_midsteps(simulation_result)
df

In [ ]:
df.columns

In [ ]:
# create target price scale
liquidation_ratio = 1.45
df['target_price_scaled'] = df['target_price'] * liquidation_ratio

### Generate key plots for analysis

In [ ]:
sns.lineplot(data=df,x="timestamp", y="eth_price",label='Generated Eth price')
plt.ylim(250, 600)
ax2 = plt.twinx()
sns.lineplot(data=df,x="timestamp", y="market_price",ax=ax2,color='r',label='Market Price in Rai')
sns.lineplot(data=df,x="timestamp", y="target_price_scaled",ax=ax2,color='g',label='Redemption Price in Rai')
plt.title('Generated Eth price vs Simulation Market and Redemption Prices')
plt.legend(loc="upper left")
plt.show()

In [ ]:
sns.lineplot(data=df,x="timestamp", y="eth_locked",label='Total Eth locked')
plt.legend(loc="upper left")
plt.show()

In [ ]:
sns.lineplot(data=df,x="timestamp", y="target_rate",label='Redemption Rate')
plt.legend(loc="upper left")
plt.show()

In [ ]:
sns.lineplot(data=df,x="timestamp", y="ETH_balance",label='Eth Balance')
plt.legend(loc="upper left")
plt.show()

In [ ]:
sns.lineplot(data=df,x="timestamp", y="RAI_balance",label='Rai Balance')
plt.legend(loc="upper left")
plt.show()

## Conclusion